# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface = await new_transport_interface("0.0.0.0", 0)
client_tiface.run()

server_tiface = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface.run()

15:38:18.177470 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
15:38:18.178259 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
15:38:18.178276 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
15:38:18.178896 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
15:38:18.178904 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7faf90b9e990>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

import datetime
t = lambda: datetime.datetime.now().strftime("%H:%M:%S.%f")


def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{t()} {message}{CRESET}", flush=True)
 
#cprint = lambda tag, message: None    # Uncomment this to disable Python logging
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
{"c": client_tiface, "s": server_tiface}

{'c': TransportInterface{ transport: None,   cmd_tx: ...,   response_rx: ...,   notify_rx: ... },
 's': TransportInterface{ transport: None,   cmd_tx: ...,   response_rx: ...,   notify_rx: ... }}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7faf90c29480>,
 's': <FilterInterface at 0x7faf90c295a0>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

15:38:18.230686 [INFO ] - [F] About to send pings to servers: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server (both our local and Chococat) as ping endpoints

In [10]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

15:38:18.431523 [INFO ] - [F] About to send pings to servers: []


'157.230.134.224'

# !!! COMMENTED OUT server_ept BELOW !!!

In [11]:
#server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[choco_ept])  # endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(157.230.134.224:2016)] }

In [12]:
await client_fiface.command_response(filter_cmd)

15:38:18.600504 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(157.230.134.224:2016)] }


Accepted

### Here be dragons

Unblock the Python async event loop by running some random async code. IDK why?

**Anyway,** I just copied this shit from StackOverflow with slight modifications for Jupyter (event loop already running): https://stackoverflow.com/a/53420574

In [13]:
import time
import asyncio

async def sleeper():
    print(f'Time: {time.time() - start:.2f}')
    await asyncio.sleep(1)
async def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        await sleeper()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()
loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(sum("A", [1, 2])),
    loop.create_task(sum("B", [1, 2, 3])),
]
done, pending = await asyncio.wait(tasks)
assert len(pending) == 0

end = time.time()
print(f'Time: {end-start:.2f} sec')

Task A: Computing 0+1
Time: 0.00
Task B: Computing 0+1
Time: 0.00
15:38:18.630645 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:18.831482 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:19.031097 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:19.231218 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:19.430522 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
Task A: Computing 1+2
Time: 1.00
Task B: Computing 1+2
Time: 1.00
15:38:19.630884 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:19.830959 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:20.031869 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:20.230564 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:20.4311

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

In [14]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

15:38:21.631015 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:21.830628 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:22.031787 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:22.230789 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:22.430851 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:22.631390 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:22.830523 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:23.030574 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:23.230779 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:23.431327 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:23.631506 [INFO ] - [F] About to send pings 

[]

In [15]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

Sleeping again....

In [16]:
time.sleep(10.0)
client_notifications = client_fiface.get_notifications()
client_notifications

15:38:24.631475 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:24.830860 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:25.030801 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:25.230851 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:25.431780 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:25.631641 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:25.830674 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:26.031573 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:26.231353 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:26.430783 [INFO ] - [F] About to send pings to servers: [Endpoint(157.230.134.224:2016)]
15:38:26.630850 [INFO ] - [F] About to send pings 

[]

In [17]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

15:38:34.658105 [TRACE] - [F<-A,C] New command: ClearPingEndpoints


Accepted

15:38:34.658117 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(157.230.134.224:2016)]


In [18]:
client_notifications = client_fiface.get_notifications()
client_notifications

[]

### Now shutdown

In [19]:
time.sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))

15:38:34.831417 [INFO ] - [F] About to send pings to servers: []
15:38:35.030839 [INFO ] - [F] About to send pings to servers: []
15:38:35.230516 [INFO ] - [F] About to send pings to servers: []
15:38:35.430929 [INFO ] - [F] About to send pings to servers: []
15:38:35.630962 [INFO ] - [F] About to send pings to servers: []
15:38:35.690365 [TRACE] - [F<-A,C] New command: Shutdown { graceful: true }
15:38:35.690383 [INFO ] - [F] shutting down
